In [13]:
import nltk
nltk.download('wordnet')
import spacy
import re
import pandas as pd
nlp = spacy.load('en_core_web_sm', disable=['ner', 'textcat'])
from nltk.stem.wordnet import WordNetLemmatizer


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def clean(doc):
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [WordNetLemmatizer().lemmatize(token.text) for token in tokens]

    return " ".join(final_token)

# function to preprocess speech
# def clean(text):

#     # removing paragraph numbers
#     text = re.sub('[0-9]+.\t', '', str(text))
#     # removing new line characters
#     text = re.sub('\n ', '', str(text))
#     text = re.sub('\n', ' ', str(text))
#     # removing apostrophes
#     text = re.sub("'s", '', str(text))
#     # removing hyphens
#     text = re.sub("-", ' ', str(text))
#     text = re.sub("— ", '', str(text))
#     # removing quotation marks
#     text = re.sub('\"', '', str(text))
#     # removing salutations
#     text = re.sub("Mr\.", 'Mr', str(text))
#     text = re.sub("Mrs\.", 'Mrs', str(text))
#     # removing any reference to outside text
#     text = re.sub("[\(\[].*?[\)\]]", "", str(text))

#     return text


In [15]:
# split sentences
def sentences(text):
    # split sentences and questions
    text = re.split('[.?]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent


In [16]:
# Loading dataset 
df = pd.read_csv('Alldata_refined.csv')
# Droping NULL rows
df = df.dropna()
# Extracting location col
df = df["Locations"]
# Converting Data frame to sorted list in lower case
Data_of_region = df.values.tolist()
Data_of_region = [each_city.lower() for each_city in Data_of_region]
Data_of_region = list(dict.fromkeys(sorted(Data_of_region)))
# Storing indexes of each alphabet starting index 
index = dict()
# Helping variables to store indexes 
flag= False
push = False
current_alphabet = ""
start = 0
finish = 0
# Creating index hash 
for i in range(len(Data_of_region)):
    if i != 0 and Data_of_region[i][0] != Data_of_region[i][0]:
        flag = True
        push = True
        finish = i-1
    if push == True:
        index[current_alphabet] = finish
    if flag == False:
        start = i
        current_alphabet = Data_of_region[i][0]
        index.__setitem__(current_alphabet, i)


In [17]:
def Get_location(read_more,header):
    header = header.lower()
    header = clean(header)
    header = header.split()
    text = sentences(clean(read_more))
    cities = dict()
    for i in text:
        doc = nlp(i)
        for token in range(len(doc)):
                if doc[token].pos_ == "PROPN":
                    flag = False
                    end = index[doc[token].text.lower()[0]]
                    if end == index['a']:
                        start = 0
                    else:
                        start = chr(ord(doc[token].text.lower()[0])-1)
                        # print(start)
                        while(True):
                            if start in index: 
                                start = index[start]-1
                                break
                            else:
                                start = chr(ord(start)-1)
                    area_count = 0
                    for areas in range(start,end):
                        words = Data_of_region[areas].split()
                        subtoken = token
                        checker = []
                        for iterator in range(len(words)):
                            if subtoken + iterator < len(doc):
                                if doc[subtoken+iterator].text.lower() == words[iterator]:
                                    checker.append(1)
                        if len(checker) == len(words):
                            city = ' '.join(words)
                            area_count = len(words[iterator])
                            flag = True
                            break
                    if flag:
                        match = False
                        word1 = ""
                        word2 = ""
                        if token != 0:
                            word1 = doc[token-1].text.lower()
                        subtoken = token + area_count
                        if subtoken + 1 < len(doc):
                            word2 = doc[subtoken+1].text.lower()
                        if word1 in header or word2 in header:
                            match = True
                        if city in cities:
                                if match:
                                    cities[city] += 3
                                else:
                                    cities[city] += 1
                        else:
                            if match:
                                cities.__setitem__(city, 3)
                            else:
                                cities.__setitem__(city, 1)
    return cities


In [19]:
header = "DADU / ISLAMABAD: Sindh braced for yet more flooding on Thursday as a surge of water flowed down the Indus river, leaving parts of Dadu district inundated."
# header="123"
print(Get_location(open("text.txt", "r", encoding='utf8').read(),header))



{'mehar': 2, 'balochistan': 1, 'khyber pakhtunkhwa': 1, 'dadu': 2, 'khairpur': 2, 'johi': 1, 'jamali': 1, 'islamabad': 1}
